## NYC Property Price Prediction

Given *data about property in New York City*, let's try to predict the **price** of a given piece of property.

We will use XGBoost to make our predictions.

Data source: https://www.kaggle.com/datasets/new-york-city/nyc-property-sales

### Importing Libraries

In [5]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import xgboost as xgb
from sklearn.metrics import r2_score

In [6]:
data = pd.read_csv('nyc-rolling-sales.csv')
data

,Unnamed: 0,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,,C2,153 AVENUE B,,10009,5,0,5,1633,6440,1900,2,C2,6625000,2017-07-19 00:00:00
1,5,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,26,,C7,234 EAST 4TH STREET,,10009,28,3,31,4616,18690,1900,2,C7,-,2016-12-14 00:00:00
2,6,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,39,,C7,197 EAST 3RD STREET,,10009,16,1,17,2212,7803,1900,2,C7,-,2016-12-09 00:00:00
3,7,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,,C4,154 EAST 7TH STREET,,10009,10,0,10,2272,6794,1913,2,C4,3936272,2016-09-23 00:00:00
4,8,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,,C2,301 EAST 10TH STREET,,10009,6,0,6,2369,4615,1900,2,C2,8000000,2016-11-17 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84543,8409,5,WOODROW,02 TWO FAMILY DWELLINGS,1,7349,34,,B9,37 QUAIL LANE,,10309,2,0,2,2400,2575,1998,1,B9,450000,2016-11-28 00:00:00
84544,8410,5,WOODROW,02 TWO FAMILY DWELLINGS,1,7349,78,,B9,32 PHEASANT LANE,,10309,2,0,2,2498,2377,1998,1,B9,550000,2017-04-21 00:00:00
84545,8411,5,WOODROW,02 TWO FAMILY DWELLINGS,1,7351,60,,B2,49 PITNEY AVENUE,,10309,2,0,2,4000,1496,1925,1,B2,460000,2017-07-05 00:00:00
84546,8412,5,WOODROW,22 STORE BUILDINGS,4,7100,28,,K6,2730 ARTHUR KILL ROAD,,10309,0,7,7,208033,64117,2001,4,K6,11693337,2016-12-21 00:00:00


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84548 entries, 0 to 84547
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Unnamed: 0                      84548 non-null  int64 
 1   BOROUGH                         84548 non-null  int64 
 2   NEIGHBORHOOD                    84548 non-null  object
 3   BUILDING CLASS CATEGORY         84548 non-null  object
 4   TAX CLASS AT PRESENT            84548 non-null  object
 5   BLOCK                           84548 non-null  int64 
 6   LOT                             84548 non-null  int64 
 7   EASE-MENT                       84548 non-null  object
 8   BUILDING CLASS AT PRESENT       84548 non-null  object
 9   ADDRESS                         84548 non-null  object
 10  APARTMENT NUMBER                84548 non-null  object
 11  ZIP CODE                        84548 non-null  int64 
 12  RESIDENTIAL UNITS               84548 non-null

### Preprocessing

In [8]:
df = data.copy()

In [9]:
df['SALE PRICE'].unique()

array(['6625000', ' -  ', '3936272', ..., '408092', '11693337', '69300'],
      dtype=object)

In [13]:
df['SALE PRICE'] = df['SALE PRICE'].replace(' -  ', np.NaN).astype(float)

In [16]:
df['SALE PRICE'].isna().mean()

0.17222169655107158

In [17]:
# Remove any records we do not have SALE PRICE for
df = df.dropna(axis=0).reset_index(drop=True)

In [19]:
df

,Unnamed: 0,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,,C2,153 AVENUE B,,10009,5,0,5,1633,6440,1900,2,C2,6625000.0,2017-07-19 00:00:00
1,7,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,,C4,154 EAST 7TH STREET,,10009,10,0,10,2272,6794,1913,2,C4,3936272.0,2016-09-23 00:00:00
2,8,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,,C2,301 EAST 10TH STREET,,10009,6,0,6,2369,4615,1900,2,C2,8000000.0,2016-11-17 00:00:00
3,10,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,406,32,,C4,210 AVENUE B,,10009,8,0,8,1750,4226,1920,2,C4,3192840.0,2016-09-23 00:00:00
4,13,1,ALPHABET CITY,08 RENTALS - ELEVATOR APARTMENTS,2,387,153,,D9,629 EAST 5TH STREET,,10009,24,0,24,4489,18523,1920,2,D9,16232000.0,2016-11-07 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69982,8409,5,WOODROW,02 TWO FAMILY DWELLINGS,1,7349,34,,B9,37 QUAIL LANE,,10309,2,0,2,2400,2575,1998,1,B9,450000.0,2016-11-28 00:00:00
69983,8410,5,WOODROW,02 TWO FAMILY DWELLINGS,1,7349,78,,B9,32 PHEASANT LANE,,10309,2,0,2,2498,2377,1998,1,B9,550000.0,2017-04-21 00:00:00
69984,8411,5,WOODROW,02 TWO FAMILY DWELLINGS,1,7351,60,,B2,49 PITNEY AVENUE,,10309,2,0,2,4000,1496,1925,1,B2,460000.0,2017-07-05 00:00:00
69985,8412,5,WOODROW,22 STORE BUILDINGS,4,7100,28,,K6,2730 ARTHUR KILL ROAD,,10309,0,7,7,208033,64117,2001,4,K6,11693337.0,2016-12-21 00:00:00


In [25]:
{column: len(df[column].unique()) for column in df.columns}

{'Unnamed: 0': 25934,
 'BOROUGH': 5,
 'NEIGHBORHOOD': 254,
 'BUILDING CLASS CATEGORY': 47,
 'TAX CLASS AT PRESENT': 11,
 'BLOCK': 10848,
 'LOT': 2484,
 'EASE-MENT': 1,
 'BUILDING CLASS AT PRESENT': 161,
 'ADDRESS': 57304,
 'APARTMENT NUMBER': 3320,
 'ZIP CODE': 184,
 'RESIDENTIAL UNITS': 158,
 'COMMERCIAL UNITS': 43,
 'TOTAL UNITS': 169,
 'LAND SQUARE FEET': 5293,
 'GROSS SQUARE FEET': 5095,
 'YEAR BUILT': 155,
 'TAX CLASS AT TIME OF SALE': 4,
 'BUILDING CLASS AT TIME OF SALE': 161,
 'SALE PRICE': 10007,
 'SALE DATE': 362}

In [26]:
# Remove unnecessary/difficult feature columns
df = df.drop(['Unnamed: 0', 'BLOCK', 'LOT', 'ADDRESS', 'EASE-MENT', 'APARTMENT NUMBER'], axis=1)

In [27]:
df

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BUILDING CLASS AT PRESENT,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,C2,10009,5,0,5,1633,6440,1900,2,C2,6625000.0,2017-07-19 00:00:00
1,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,C4,10009,10,0,10,2272,6794,1913,2,C4,3936272.0,2016-09-23 00:00:00
2,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,C2,10009,6,0,6,2369,4615,1900,2,C2,8000000.0,2016-11-17 00:00:00
3,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,C4,10009,8,0,8,1750,4226,1920,2,C4,3192840.0,2016-09-23 00:00:00
4,1,ALPHABET CITY,08 RENTALS - ELEVATOR APARTMENTS,2,D9,10009,24,0,24,4489,18523,1920,2,D9,16232000.0,2016-11-07 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69982,5,WOODROW,02 TWO FAMILY DWELLINGS,1,B9,10309,2,0,2,2400,2575,1998,1,B9,450000.0,2016-11-28 00:00:00
69983,5,WOODROW,02 TWO FAMILY DWELLINGS,1,B9,10309,2,0,2,2498,2377,1998,1,B9,550000.0,2017-04-21 00:00:00
69984,5,WOODROW,02 TWO FAMILY DWELLINGS,1,B2,10309,2,0,2,4000,1496,1925,1,B2,460000.0,2017-07-05 00:00:00
69985,5,WOODROW,22 STORE BUILDINGS,4,K6,10309,0,7,7,208033,64117,2001,4,K6,11693337.0,2016-12-21 00:00:00


In [29]:
df['LAND SQUARE FEET'] = df['LAND SQUARE FEET'].replace(' -  ', np.NaN)
df['GROSS SQUARE FEET'] = df['GROSS SQUARE FEET'].replace(' -  ', np.NaN)

In [31]:
df.isna().mean()

BOROUGH                           0.000000
NEIGHBORHOOD                      0.000000
BUILDING CLASS CATEGORY           0.000000
TAX CLASS AT PRESENT              0.000000
BUILDING CLASS AT PRESENT         0.000000
ZIP CODE                          0.000000
RESIDENTIAL UNITS                 0.000000
COMMERCIAL UNITS                  0.000000
TOTAL UNITS                       0.000000
LAND SQUARE FEET                  0.302742
GROSS SQUARE FEET                 0.310615
YEAR BUILT                        0.000000
TAX CLASS AT TIME OF SALE         0.000000
BUILDING CLASS AT TIME OF SALE    0.000000
SALE PRICE                        0.000000
SALE DATE                         0.000000
dtype: float64

In [32]:
# Fill missing values with column means
for column in ['LAND SQUARE FEET', 'GROSS SQUARE FEET']:
    df[column] = df[column].astype(float)
    df[column] = df[column].fillna(df[column].mean())

In [33]:
df.isna().sum().sum()

0

In [34]:
df

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BUILDING CLASS AT PRESENT,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,C2,10009,5,0,5,1633.0,6440.0,1900,2,C2,6625000.0,2017-07-19 00:00:00
1,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,C4,10009,10,0,10,2272.0,6794.0,1913,2,C4,3936272.0,2016-09-23 00:00:00
2,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,C2,10009,6,0,6,2369.0,4615.0,1900,2,C2,8000000.0,2016-11-17 00:00:00
3,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,C4,10009,8,0,8,1750.0,4226.0,1920,2,C4,3192840.0,2016-09-23 00:00:00
4,1,ALPHABET CITY,08 RENTALS - ELEVATOR APARTMENTS,2,D9,10009,24,0,24,4489.0,18523.0,1920,2,D9,16232000.0,2016-11-07 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69982,5,WOODROW,02 TWO FAMILY DWELLINGS,1,B9,10309,2,0,2,2400.0,2575.0,1998,1,B9,450000.0,2016-11-28 00:00:00
69983,5,WOODROW,02 TWO FAMILY DWELLINGS,1,B9,10309,2,0,2,2498.0,2377.0,1998,1,B9,550000.0,2017-04-21 00:00:00
69984,5,WOODROW,02 TWO FAMILY DWELLINGS,1,B2,10309,2,0,2,4000.0,1496.0,1925,1,B2,460000.0,2017-07-05 00:00:00
69985,5,WOODROW,22 STORE BUILDINGS,4,K6,10309,0,7,7,208033.0,64117.0,2001,4,K6,11693337.0,2016-12-21 00:00:00


In [36]:
{column: len(df[column].unique()) for column in df.select_dtypes('object').columns}

{'NEIGHBORHOOD': 254,
 'BUILDING CLASS CATEGORY': 47,
 'TAX CLASS AT PRESENT': 11,
 'BUILDING CLASS AT PRESENT': 161,
 'BUILDING CLASS AT TIME OF SALE': 161,
 'SALE DATE': 362}

In [37]:
df['SALE DATE']

0        2017-07-19 00:00:00
1        2016-09-23 00:00:00
2        2016-11-17 00:00:00
3        2016-09-23 00:00:00
4        2016-11-07 00:00:00
                ...         
69982    2016-11-28 00:00:00
69983    2017-04-21 00:00:00
69984    2017-07-05 00:00:00
69985    2016-12-21 00:00:00
69986    2016-10-27 00:00:00
Name: SALE DATE, Length: 69987, dtype: object

In [38]:
df['SALE DATE'] = pd.to_datetime(df['SALE DATE'])
df['YEAR'] = df['SALE DATE'].apply(lambda x: x.year)
df['MONTH'] = df['SALE DATE'].apply(lambda x: x.month)
df['DAY'] = df['SALE DATE'].apply(lambda x: x.day)
df = df.drop('SALE DATE', axis=1)

In [39]:
df

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BUILDING CLASS AT PRESENT,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,YEAR,MONTH,DAY
0,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,C2,10009,5,0,5,1633.0,6440.0,1900,2,C2,6625000.0,2017,7,19
1,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,C4,10009,10,0,10,2272.0,6794.0,1913,2,C4,3936272.0,2016,9,23
2,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,C2,10009,6,0,6,2369.0,4615.0,1900,2,C2,8000000.0,2016,11,17
3,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,C4,10009,8,0,8,1750.0,4226.0,1920,2,C4,3192840.0,2016,9,23
4,1,ALPHABET CITY,08 RENTALS - ELEVATOR APARTMENTS,2,D9,10009,24,0,24,4489.0,18523.0,1920,2,D9,16232000.0,2016,11,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69982,5,WOODROW,02 TWO FAMILY DWELLINGS,1,B9,10309,2,0,2,2400.0,2575.0,1998,1,B9,450000.0,2016,11,28
69983,5,WOODROW,02 TWO FAMILY DWELLINGS,1,B9,10309,2,0,2,2498.0,2377.0,1998,1,B9,550000.0,2017,4,21
69984,5,WOODROW,02 TWO FAMILY DWELLINGS,1,B2,10309,2,0,2,4000.0,1496.0,1925,1,B2,460000.0,2017,7,5
69985,5,WOODROW,22 STORE BUILDINGS,4,K6,10309,0,7,7,208033.0,64117.0,2001,4,K6,11693337.0,2016,12,21


In [40]:
# Make numerical categorical features into string columns
for column in ['BOROUGH', 'ZIP CODE']:
    df[column] = df[column].astype(str)

In [41]:
{column: len(df[column].unique()) for column in df.select_dtypes('object').columns}

{'BOROUGH': 5,
 'NEIGHBORHOOD': 254,
 'BUILDING CLASS CATEGORY': 47,
 'TAX CLASS AT PRESENT': 11,
 'BUILDING CLASS AT PRESENT': 161,
 'ZIP CODE': 184,
 'BUILDING CLASS AT TIME OF SALE': 161}

In [42]:
# Encode nominal features
def onehot_encode(df, columns, prefixes):
    df = df.copy()
    for column, prefix in zip(columns, prefixes):
        dummies = pd.get_dummies(df[column], prefix=prefix, dtype=int)
        df = pd.concat([df, dummies], axis=1)
        df = df.drop(column, axis=1)
    return df

In [44]:
nominal_features = ['BOROUGH', 'NEIGHBORHOOD', 'BUILDING CLASS CATEGORY', 'TAX CLASS AT PRESENT', 'BUILDING CLASS AT PRESENT', 'ZIP CODE', 'BUILDING CLASS AT TIME OF SALE']
PREFIXES = ['BR', 'NB', 'BCC', 'TCP', 'BCP', 'ZC', 'BCTS']

df = onehot_encode(df, columns=nominal_features, prefixes=PREFIXES)
df

,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,SALE PRICE,YEAR,MONTH,DAY,BR_1,BR_2,BR_3,BR_4,BR_5,NB_AIRPORT LA GUARDIA,NB_ALPHABET CITY,NB_ANNADALE,NB_ARDEN HEIGHTS,NB_ARROCHAR,NB_ARROCHAR-SHORE ACRES,NB_ARVERNE,NB_ASTORIA,NB_BATH BEACH,NB_BATHGATE,NB_BAY RIDGE,NB_BAYCHESTER,NB_BAYSIDE,NB_BEDFORD PARK/NORWOOD,NB_BEDFORD STUYVESANT,NB_BEECHHURST,NB_BELLE HARBOR,NB_BELLEROSE,NB_BELMONT,NB_BENSONHURST,NB_BERGEN BEACH,NB_BLOOMFIELD,NB_BOERUM HILL,NB_BOROUGH PARK,NB_BRIARWOOD,NB_BRIGHTON BEACH,NB_BROAD CHANNEL,NB_BRONX PARK,NB_BRONXDALE,NB_BROOKLYN HEIGHTS,NB_BROWNSVILLE,NB_BULLS HEAD,NB_BUSH TERMINAL,NB_BUSHWICK,NB_CAMBRIA HEIGHTS,NB_CANARSIE,NB_CARROLL GARDENS,NB_CASTLE HILL/UNIONPORT,NB_CASTLETON CORNERS,NB_CHELSEA,NB_CHINATOWN,NB_CITY ISLAND,NB_CITY ISLAND-PELHAM STRIP,NB_CIVIC CENTER,NB_CLINTON,NB_CLINTON HILL,NB_CLOVE LAKES,NB_CO-OP CITY,NB_COBBLE HILL,NB_COBBLE HILL-WEST,NB_COLLEGE POINT,NB_CONCORD,NB_CONCORD-FOX HILLS,NB_CONEY ISLAND,NB_CORONA,NB_COUNTRY CLUB,NB_CROTONA PARK,NB_CROWN HEIGHTS,NB_CYPRESS HILLS,NB_DONGAN HILLS,NB_DONGAN HILLS-COLONY,NB_DONGAN HILLS-OLD TOWN,NB_DOUGLASTON,NB_DOWNTOWN-FULTON FERRY,NB_DOWNTOWN-FULTON MALL,NB_DOWNTOWN-METROTECH,NB_DYKER HEIGHTS,NB_EAST ELMHURST,NB_EAST NEW YORK,NB_EAST RIVER,NB_EAST TREMONT,NB_EAST VILLAGE,NB_ELMHURST,NB_ELTINGVILLE,NB_EMERSON HILL,NB_FAR ROCKAWAY,NB_FASHION,NB_FIELDSTON,NB_FINANCIAL,NB_FLATBUSH-CENTRAL,NB_FLATBUSH-EAST,NB_FLATBUSH-LEFFERTS GARDEN,NB_FLATBUSH-NORTH,NB_FLATIRON,NB_FLATLANDS,NB_FLORAL PARK,NB_FLUSHING MEADOW PARK,NB_FLUSHING-NORTH,NB_FLUSHING-SOUTH,NB_FORDHAM,NB_FOREST HILLS,NB_FORT GREENE,NB_FRESH KILLS,NB_FRESH MEADOWS,NB_GERRITSEN BEACH,NB_GLEN OAKS,NB_GLENDALE,NB_GOWANUS,NB_GRAMERCY,NB_GRANT CITY,NB_GRASMERE,NB_GRAVESEND,NB_GREAT KILLS,NB_GREAT KILLS-BAY TERRACE,NB_GREENPOINT,NB_GREENWICH VILLAGE-CENTRAL,NB_GREENWICH VILLAGE-WEST,NB_GRYMES HILL,NB_HAMMELS,NB_HARLEM-CENTRAL,NB_HARLEM-EAST,NB_HARLEM-UPPER,NB_HARLEM-WEST,NB_HIGHBRIDGE/MORRIS HEIGHTS,NB_HILLCREST,NB_HOLLIS,NB_HOLLIS HILLS,NB_HOLLISWOOD,NB_HOWARD BEACH,NB_HUGUENOT,NB_HUNTS POINT,NB_INWOOD,NB_JACKSON HEIGHTS,NB_JAMAICA,NB_JAMAICA BAY,NB_JAMAICA ESTATES,NB_JAMAICA HILLS,NB_JAVITS CENTER,NB_KENSINGTON,NB_KEW GARDENS,NB_KINGSBRIDGE HTS/UNIV HTS,NB_KINGSBRIDGE/JEROME PARK,NB_KIPS BAY,NB_LAURELTON,NB_LITTLE ITALY,NB_LITTLE NECK,NB_LIVINGSTON,NB_LONG ISLAND CITY,NB_LOWER EAST SIDE,NB_MADISON,NB_MANHATTAN BEACH,NB_MANHATTAN VALLEY,NB_MANOR HEIGHTS,NB_MARINE PARK,NB_MARINERS HARBOR,NB_MASPETH,NB_MELROSE/CONCOURSE,NB_MIDDLE VILLAGE,NB_MIDLAND BEACH,NB_MIDTOWN CBD,NB_MIDTOWN EAST,NB_MIDTOWN WEST,NB_MIDWOOD,NB_MILL BASIN,NB_MORNINGSIDE HEIGHTS,NB_MORRIS PARK/VAN NEST,NB_MORRISANIA/LONGWOOD,NB_MOTT HAVEN/PORT MORRIS,NB_MOUNT HOPE/MOUNT EDEN,NB_MURRAY HILL,NB_NAVY YARD,NB_NEPONSIT,NB_NEW BRIGHTON,NB_NEW BRIGHTON-ST. GEORGE,NB_NEW DORP,NB_NEW DORP-BEACH,NB_NEW DORP-HEIGHTS,NB_NEW SPRINGVILLE,NB_OAKLAND GARDENS,NB_OAKWOOD,NB_OAKWOOD-BEACH,NB_OCEAN HILL,NB_OCEAN PARKWAY-NORTH,NB_OCEAN PARKWAY-SOUTH,NB_OLD MILL BASIN,NB_OZONE PARK,NB_PARK SLOPE,NB_PARK SLOPE SOUTH,NB_PARKCHESTER,NB_PELHAM BAY,NB_PELHAM GARDENS,NB_PELHAM PARKWAY NORTH,NB_PELHAM PARKWAY SOUTH,NB_PLEASANT PLAINS,NB_PORT IVORY,NB_PORT RICHMOND,NB_PRINCES BAY,NB_PROSPECT HEIGHTS,NB_QUEENS VILLAGE,NB_RED HOOK,NB_REGO PARK,NB_RICHMOND HILL,NB_RICHMONDTOWN,NB_RICHMONDTOWN-LIGHTHS HILL,NB_RIDGEWOOD,NB_RIVERDALE,NB_ROCKAWAY PARK,NB_ROOSEVELT ISLAND,NB_ROSEBANK,NB_ROSEDALE,NB_ROSSVILLE,NB_ROSSVILLE-CHARLESTON,NB_ROSSVILLE-PORT MOBIL,NB_ROSSVILLE-RICHMOND VALLEY,NB_SCHUYLERVILLE/PELHAM BAY,NB_SEAGATE,NB_SHEEPSHEAD BAY,NB_SILVER LAKE,NB_SO. JAMAICA-BAISLEY PARK,NB_SOHO,NB_SOUNDVIEW,NB_SOUTH BEACH,NB_SOUTH JAMAICA,NB_SOUTH OZONE PARK,NB_SOUTHBRIDGE,NB_SPRING CREEK,NB_SPRINGFIELD GARDENS,NB_ST. ALBANS,NB_STAPLETON,NB_STAPLETON-CLIFTON,NB_SUNNYSIDE,NB_SUNSET PARK,NB_THROGS NECK,NB_TODT HILL,NB_TOMPKINSVILLE,NB_TOTTENVILLE,NB_TRAVIS,NB_TRIBECA,NB_UPPER EAST SIDE (59-79),NB_UPPER

In [45]:
# Split df into X and y
y = df['SALE PRICE'].copy()
X = df.drop('SALE PRICE', axis=1).copy()

In [46]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=1)

In [47]:
X_train.shape, X_test.shape

((48990, 833), (20997, 833))

In [48]:
# Scale X with a standard scaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train), index=X_train.index, columns=X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test), index=X_test.index, columns=X_test.columns)

In [49]:
X_train

RESIDENTIAL UNITS  COMMERCIAL UNITS  TOTAL UNITS  LAND SQUARE FEET  \
14693           0.009770         -0.017765    -0.003572         -0.038035   
46614           0.009770         -0.017765    -0.003572         -0.049029   
56179          -0.125664         -0.017765    -0.112660         -0.002562   
14690           0.009770         -0.017765    -0.003572         -0.028910   
19974           0.077487         -0.017765     0.050973         -0.046040   
...                  ...               ...          ...               ...   
49100           0.009770         -0.017765    -0.003572         -0.050683   
20609          -0.057947         -0.017765    -0.058116         -0.063385   
21440           0.009770         -0.017765    -0.003572         -0.046040   
50057          -0.057947         -0.017765    -0.058116         -0.040704   
5192           -0.057947         -0.017765    -0.058116         -0.002562   

       GROSS SQUARE FEET  YEAR BUILT  TAX CLASS AT TIME OF SALE      YEAR  \
14693          -0.057228    0.251193                  -0.828398  0.724916   
46614          -0.062939    0.251193                  -0.828398  0.724916   
56179          -0.001711    0.304859                   0.463280  0.724916   
14690          -0.073115    0.289526                  -0.828398  0.724916   
19974          -0.033653    0.318275                  -0.828398  0.724916   
...                  ...         ...                        ...       ...   
49100          -0.096508    0.354691                  -0.828398 -1.379471   
20609          -0.091346    0.299109                  -0.828398  0.724916   
21440          -0.066306    0.195611                  -0.828398 -1.379471   
50057          -0.091639    0.270360                  -0.828398 -1.379471   
5192           -0.001711    0.362358                   0.463280 -1.379471   

          MONTH       DAY      BR_1      BR_2      BR_3      BR_4      BR_5  \
14693 -1.318944 -0.725224 -0.507134  3.009908 -0.725014 -0.600663 -0.307415   
46614 -0.450014 -0.952329 -0.507134 -0.332236 -0.725014  1.664828 -0.307415   
56179 -0.160371 -0.384568 -0.507134 -0.332236 -0.725014  1.664828 -0.307415   
14690 -1.318944  1.318713 -0.507134  3.009908 -0.725014 -0.600663 -0.307415   
19974 -0.450014  0.637401 -0.507134  3.009908 -0.725014 -0.600663 -0.307415   
...         ...       ...       ...       ...       ...       ...       ...   
49100  0.708559  1.318713 -0.507134 -0.332236 -0.725014  1.664828 -0.307415   
20609 -0.450014  0.978057 -0.507134  3.009908 -0.725014 -0.600663 -0.307415   
21440  1.287846 -0.725224 -0.507134  3.009908 -0.725014 -0.600663 -0.307415   
50057  0.998202  0.183192 -0.507134 -0.332236 -0.725014  1.664828 -0.307415   
5192   1.287846 -0.611672  1.971864 -0.332236 -0.725014 -0.600663 -0.307415   

       NB_AIRPORT LA GUARDIA  NB_ALPHABET CITY  NB_ANNADALE  NB_ARDEN HEIGHTS  \
14693              -0.011067         -0.044541    -0.045452         -0.060217   
46614              -0.011067         -0.044541    -0.045452         -0.060217   
56179              -0.011067         -0.044541    -0.045452         -0.060217   
14690              -0.011067         -0.044541    -0.045452         -0.060217   
19974              -0.011067         -0.044541    -0.045452         -0.060217   
...                      ...               ...          ...               ...   
49100              -0.011067         -0.044541    -0.045452         -0.060217   
20609              -0.011067         -0.044541    -0.045452         -0.060217   
21440              -0.011067         -0.044541    -0.045452         -0.060217   
50057              -0.011067         -0.044541    -0.045452         -0.060217   
5192               -0.011067         -0.044541    -0.045452         -0.060217   

       NB_ARROCHAR  NB_ARROCHAR-SHORE ACRES  NB_ARVERNE  NB_ASTORIA  \
14693    -0.020709                -0.017501   -0.046786   -0.091981   
46614    -0.020709                -0.017501   -0.046786   -0.091981   
56179    -0.020709    

In [50]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=True, random_state=1)

In [51]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)
dtest = xgb.DMatrix(X_test, label=y_test)

In [54]:
params = {'learning_rate': 0.001, 'max_depth': 6, 'lambda': 0.01}

model = xgb.train(params, dtrain, num_boost_round=10000, evals=[(dval, 'eval')], early_stopping_rounds=10)

[0]	eval-rmse:4216137.84564
[1]	eval-rmse:4215178.81389
[2]	eval-rmse:4214247.66711
[3]	eval-rmse:4213344.09429
[4]	eval-rmse:4212468.28749
[5]	eval-rmse:4211619.99581
[6]	eval-rmse:4210799.25729
[7]	eval-rmse:4210005.89925
[8]	eval-rmse:4209240.05941
[9]	eval-rmse:4208501.45411
[10]	eval-rmse:4207790.14735
[11]	eval-rmse:4207105.93917
[12]	eval-rmse:4206448.92554
[13]	eval-rmse:4205819.02187
[14]	eval-rmse:4205216.03513
[15]	eval-rmse:4204639.97108
[16]	eval-rmse:4204090.79173
[17]	eval-rmse:4203568.39674
[18]	eval-rmse:4203072.74977
[19]	eval-rmse:4202603.73877
[20]	eval-rmse:4202161.28611
[21]	eval-rmse:4201745.30080
[22]	eval-rmse:4201355.77882
[23]	eval-rmse:4200992.67013
[24]	eval-rmse:4200655.90607
[25]	eval-rmse:4200345.23881
[26]	eval-rmse:4200060.81666
[27]	eval-rmse:4199802.42231
[28]	eval-rmse:4199570.10614
[29]	eval-rmse:4199363.62245
[30]	eval-rmse:4199183.13598
[31]	eval-rmse:4199028.34679
[32]	eval-rmse:4198899.33032
[33]	eval-rmse:4198795.97500
[34]	eval-rmse:4198718.1

In [55]:
y_true = np.array(y_test)
y_pred = model.predict(dtest)

In [56]:
print("Model R^2 Score: {:.4f}".format(r2_score(y_true, y_pred)))

Model R^2 Score: 0.0107
